In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import warnings
import scarches as sca
warnings.filterwarnings("ignore")


import sys
sys.path.append('../scripts')
%load_ext autoreload
%autoreload 2
#%load_ext lab_black

/home/daniele/miniconda3/envs/scmouse_atlas/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 captum (see https://github.com/pytorch/captum).


In [ ]:
adata = sc.read_h5ad('/mnt/storage/Daniele/atlases/mouse/14_mouse_final_annotation.h5ad')

In [3]:
adata_manual = adata[:, adata.var['manual_gene']].copy()

In [ ]:
batch_key = 'donor_id'
celltype_key = 'Level_4_knn'

In [6]:
# hotfix
adata_manual.obs[batch_key] = adata_manual.obs[batch_key].astype(str).astype('category')
sca.models.SCVI.setup_anndata(adata_manual, layer='binned_data', batch_key=batch_key, labels_key=celltype_key)


In [7]:
vae = sca.models.SCVI(
    adata_manual,
    n_layers=2,
    encode_covariates=True,
    deeply_inject_covariates=False,
    use_layer_norm="both",
    use_batch_norm="none",
)

In [ ]:
vae.train(max_epochs=50)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 4/50:   6%|▌         | 3/50 [02:18<35:48, 45.71s/it, v_num=1, train_loss_step=2.81e+3, train_loss_epoch=2.64e+3]

In [ ]:
scanvae = sca.models.SCANVI.from_scvi_model(vae, unlabeled_category = "Unknown")
scanvae.train(max_epochs=10)

In [ ]:
adata_manual.obs['predictions'] = scanvae.predict()
print("Acc: {}".format(np.mean(adata_manual.obs.predictions == adata_manual.obs.Level_4_knn)))

In [ ]:
adata_manual.obsm['scANVI_emb_final'] = scanvae.get_latent_representation(adata_manual)

In [ ]:
adata.obsm['scANVI_emb_final'] = adata_manual.obsm['scANVI_emb_final'].copy()

In [ ]:
from sklearn_ann.kneighbors.annoy import AnnoyTransformer
sc.pp.neighbors(adata, transformer=AnnoyTransformer(15), use_rep='scANVI_emb_final')

In [ ]:
sc.tl.umap(adata, min_dist=0.25)

In [ ]:
adata.write_h5ad('/mnt/storage/Daniele/atlases/mouse/15_mouse_final_integration.h5ad')

In [ ]:
scanvae.save('/home/daniele/Code/github_synced/PDAC/models/mouse/scANVI', overwrite=True)

# create clean low level annotations

In [ ]:
adata.obs.Level_4_knn.replace('Malignant Cell - Hihgly Invasive', 'Malignant Cell - Highly Invasive', inplace=True)
adata.obs.Level_4_knn.replace('Acinar idlling', 'Acinar Idling', inplace=True)

In [ ]:
adata.obs['Level_4_final'] = adata.obs['Level_4_knn'].copy()

In [19]:
celltype_df = pd.read_csv('../../../supplementary_data/celltype_annotation.csv')
dict_level4_to_level3 = dict(zip(celltype_df['Level_4'], celltype_df['Level_3']))
dict_level3_to_level2 = dict(zip(celltype_df['Level_3'], celltype_df['Level_2']))
dict_level2_to_level1 = dict(zip(celltype_df['Level_2'], celltype_df['Level_1']))

In [20]:
adata.obs['Level_4_final'] = adata.obs['Level_4_final'].astype('category')
adata.obs['Level_3_final'] = adata.obs['Level_4_final'].replace(dict_level4_to_level3).astype('category')
adata.obs['Level_2_final'] = adata.obs['Level_3_final'].replace(dict_level3_to_level2).astype('category')
adata.obs['Level_1_final'] = adata.obs['Level_2_final'].replace(dict_level2_to_level1).astype('category')


In [25]:
adata.write_h5ad('/mnt/storage/Daniele/atlases/mouse/15_mouse_final_integration.h5ad')